## Queries on CVE records for the extraction of IoT related referenced repositories:

In [77]:
import collections
import pandas as pd
from matplotlib import pyplot as plt
import json 
import ast
import re
import os
import csv
import subprocess
import requests
import tempfile
from io import BytesIO, StringIO
from zipfile import ZipFile
from guesslang import Guess

In [78]:
df = pd.read_csv('../data/cve-records.csv')
des_str = df['description'][0]

/var/folders/p_/9ryggq9s67zbhxhzlms6wmzw0000gn/T/ipykernel_37583/2247889250.py:1: DtypeWarning: Columns (6,7,8,9,10,11,12,13,14,15,16,17,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/cve-records.csv')


### Search Query: 
"Internet of Things" OR "IoT" OR "Industry 4.0" OR "smart cities" OR "smart city"OR "smart contract" OR "manufacturing" OR "energy" OR "supply chain" 

In [79]:
def get_description(des_str):
    if des_str!=None or des_str!='':
        des_arr_dict = ast.literal_eval(des_str)
        des_cve = ""    #description of a CVE-> 'value' from array of dict.

        for dic in des_arr_dict:
            des_cve = des_cve + dic['value']
        return des_cve
        
    else:
        print('Empty description for CVE: ')
        return 0

def get_iot_cves(df):
    iot_set = ["Internet of Things", "IoT", "Industry 4.0", 
                "smart cities", "smart city", "smart contract", 
                "manufacturing", "energy", "supply chain", "orange pi", "banana pi", "arduino"]
    iot_cves = []

    for row in range(len(df)):
        des_cve = get_description(df['description'][row])
        
        # print if they are IoT related descriptions
        for x in iot_set:
            if x.lower() in des_cve.lower():
                # print(des_cve)
                # print(df['cve_id'][row])
                iot_cves.append(df['cve_id'][row])
                # print(df['reference_json'][row])
                # print('\n')
    return iot_cves

iot_cves = get_iot_cves(df)
print('count_cves:', len(iot_cves))

count_cves: 2262


In [80]:
df_iot = df[df.cve_id.isin(iot_cves)]
len(df_iot)

2240

In [81]:
iot_vcs = ['github', 'bitbucket', 'gitlab']
vcs_list = []

for ref_str in df_iot.reference_json:
    url_dict  = ast.literal_eval(ref_str)
    
    if len(url_dict) > 0:
        for ref in url_dict:
            vcs_list.append(ref['url'])     

## Vulnerabilty reporting databases and number of their occurances in CVEs

In [82]:
url_freq = collections.Counter(url_heads)
df_url = pd.DataFrame(url_freq.items(), columns=['urls', 'count'])
df_url = df_url.sort_values(by=['count'], ascending=False)
df_url.to_csv('../result/top-databases.csv', index=False, sep=';')
df_url.head(5)

NameError: name 'url_heads' is not defined

# Analysis of Infer output (report.json) file:

In [ ]:
import pandas as pd
import json
import os
import subprocess as sub
import time 

############################ Applying infer tool ############################

def json2df(file) -> pd.DataFrame:
    df = pd.DataFrame()
    with open(file) as f:
        data = json.load(f)
        df = pd.DataFrame(data)
    return df

def apply_infer(fname) -> pd.DataFrame:
    """find flaws in the file using infer tool"""
    infer_dir = 'infer-output'
    compiler = 'gcc'

    cmd = f"infer run --results-dir {infer_dir} -- {compiler} -c "
    out_file = f"{infer_dir}/report.json" # output file generated by infer tool

    df = pd.DataFrame() # dataframe to store the results

    if os.path.isfile(fname):
        cmd =  cmd + fname
        process = sub.Popen(
            cmd,
            shell=True,
            stdout=sub.PIPE,
        )
        process.wait() # wait for the process to finish
        
        # check if the output file is generated
        if os.path.isfile(out_file):
            df = json2df(out_file)
        if len(df)!=0:
            df["tool"] = "infer"
        df = df.reset_index(drop=True)

    else:
        print(f'Invalid command for infer tool! \
            \nPlease check the command again! \ncommand: {cmd}')
    return df

# fname = '../data/projects/contiki-2.4/tools/tunslip.c'
fname = '../data/projects/contiki-2.4/core/sys/timetable.c'

apply_infer(fname)

# Encoding target label 

In [22]:
import pickle 
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer

def encode_multiclass(y):
    """encode multiclass target """
    encoder = LabelEncoder()
    encoder.fit(y)
    encoded_y = encoder.transform(y)
    with open('classes.pkl', 'wb') as f:
        pickle.dump(encoder, f)
    return encoded_y

def decode_multiclass(encoded_y):
    """decode multiclass target """
    with open('classes.pkl', 'rb') as f:
        encoder = pickle.load(f)

    decoded_y = encoder.inverse_transform(encoded_y)
    # decoded_y = [x[0] for x in decoded_y]
    return decoded_y

y = ['Benign', 'CWE-120', 'Benign', 'Benign', 'CWE-120', 'CWE-20', 'CWE-19']

# target representation for binary classification
y = [x if x=='Benign' else 'Vulnerable' for x in y]

encoded_y = encode_multiclass(y)
print(y)
print(encoded_y)
print(decode_multiclass([0, 1, 1]))

['Benign', 'Vulnerable', 'Benign', 'Benign', 'Vulnerable', 'Vulnerable', 'Vulnerable']
[0 1 0 0 1 1 1]
['Benign' 'Vulnerable' 'Vulnerable']


In [20]:
y = [x if x=='Benign' else 'Vulnerable' for x in y]

['Benign',
 'Vulnerable',
 'Benign',
 'Benign',
 'Vulnerable',
 'Vulnerable',
 'Vulnerable']

In [11]:
import numpy as np 

(decode_multiclass([0, 1, 2]))

array(['Benign', 'CWE-120', 'CWE-19'], dtype='<U7')

In [3]:
import tensorflow as tf

m1 = tf.keras.metrics.Accuracy()
acc = m1.update_state([[1], [2]], [[0], [2]])
acc

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=2.0>

In [25]:
import pandas as pd 

df = pd.read_csv('../data/iDetect_refine/DNN-binary.csv')
df

,code,label
0,( strlen ( me ),1
1,*pBuffer = MQTT_PACKET_TYPE_CONNECT ; pBuffer...,1
2,*pBuffer1 = MQTT_PACKET_TYPE_CONNECT1 ; pBuff...,1
3,ActualFreq 1 = cc1ComputeFreq ( DesiredFreq1 ) ;,1
4,ActualFreq = cc1000ComputeFreq ( DesiredFreq ) ;,1
...,...,...
4715,"xResultr = xClass ( xSession , ( char * ) pxLa...",1
4716,"xResultt = xClass ( xSession , ( char * ) pxLa...",1
4717,"xResultw = xClass ( xSession , ( char * ) pxLa...",1
4718,"xResultx = C_Create ( xSessionx, ( A ) & D, si...",1


In [6]:
df = pd.read_csv('../data/TinyVul-v2-statement-multiclass.csv')
df

,code,label
0,~tx_mask & 0xFF);,Benign
1,~sin->sin_addr.s_addr);,Benign
2,~sd->gpio[0] << 8);,Benign
3,~p_sb_attn_sw->known_attn;,CWE-190
4,~oparg);,Benign
...,...,...
982578,!!(cfg->priv_flags & BIT(AQ_HW_LOOPBACK_DMA_NE...,Benign
982579,!!(addr_ctrl & SDRAM_ADDR_CTRL_ADDR_SEL_MASK(i));,Benign
982580,!!(accept_filter & QED_ACCEPT_UCAST_UNMATCHED)));,Benign
982581,!!(accept_filter & QED_ACCEPT_NONE));,Benign


In [30]:
from sklearn.preprocessing import LabelEncoder
import pickle

le = LabelEncoder()
le.fit_transform(df.label)

array([36, 16, 16, ..., 28,  3,  3])

In [40]:
def encode_multiclass_target(y):
    """encode multiclass target """
    encoder = LabelEncoder()
    encoder.fit_transform(y)
    with open('../data/classes.pkl', 'wb') as f:
        pickle.dump(encoder, f)

def decode_multiclass_target(encoded_y):
    """decode multiclass target """
    with open('../data/classes.pkl', 'rb') as f:
        encoder = pickle.load(f)

    decoded_y = encoder.inverse_transform(encoded_y)
    # decoded_y = [x[0] for x in decoded_y]
    return decoded_y

encode_multiclass_target(y)
decode_multiclass_target([0, 1, 2])

array([0, 1, 2])